## Idea:
Given $k$, we create the training set as:
Suppose store s was open on day1, day2, ...., day(k+1),
the corresponding y is the visitors of s on day(k+1),
the corresponding x is: day1 info, day1 distance to day(k+1), day2 info, day2 distance to day(k+1) ....,


In [1]:
import os
import numpy as np
import pandas as pd
import platform
import pickle
from pathlib import Path
from datetime import datetime, date, timedelta
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
%qtconsole

In [3]:
if platform.system() == 'Linux':
    data_dir = '/home/alin/Data/Recruit_Holding'
else:
    data_dir = 'C:/Users/alin/Documents/Data/Recruit_Holding'

In [4]:
DFS_dump = data_dir + '/DFS.p'
if Path(DFS_dump).is_file():
    print('load previous dump')
    DFS = pickle.load(open(DFS_dump, 'rb'))
    air_reserve = DFS['air_reserve']
    air_reserve_day = DFS['air_reserve_day']
    hpg_reserve = DFS['hpg_reserve']
    hpg_reserve_day = DFS['hpg_reserve_day']
    air_visit_hist = DFS['air_visit_hist']
    date_info = DFS['date_info']
    test = DFS['test']
    air_store_info = DFS['air_store_info']
    hpg_store_info = DFS['hpg_store_info']
    store_id_relation = DFS['store_id_relation']
    test = DFS['test']
else:
    print('run EDA1 first')

load previous dump


### Build the training and testing datasets before label encoding

### step 0: add reserve data to air_visit_hist

In [6]:
air_reserve_day1 = air_reserve_day[['air_store_id', 'day_ind', 'reserve_visitors']]
hpg_reserve_day1 = hpg_reserve_day[['hpg_store_id', 'day_ind', 'reserve_visitors']]

air_visit_hist = air_visit_hist.merge(air_reserve_day1, on=['air_store_id', 'day_ind'], how='left')

air_visit_hist = air_visit_hist.merge(hpg_reserve_day1, on=['hpg_store_id',  'day_ind'], how='left', suffixes=['_air', '_hpg'])

air_visit_hist.fillna({'reserve_visitors_air': 0, 'reserve_visitors_hpg': 0}, inplace=True)

air_visit_hist['reserve_visitors'] = air_visit_hist.apply(lambda r: r['reserve_visitors_air'] + r['reserve_visitors_hpg'], axis=1)

air_visit_hist.drop(['month_ind', 'latitude', 'longitude', 'reserve_visitors_air', 'reserve_visitors_hpg'], axis=1, inplace=True)

### step 1: sort by air_store_id and day_ind

In [9]:
air_visit_hist.sort_values(by=['air_store_id', 'day_ind'], inplace=True, ascending=False)

In [32]:
air_visit_hist.reset_index(inplace=True)
air_visit_hist.drop('index', axis=1, inplace=True)
air_visit_hist['ind'] = air_visit_hist.index

### step 2: create training set

In [63]:
a = air_visit_hist.copy()
b = air_visit_hist.copy()
k = 20
index_cols = ['air_store_id', 'ind']
cols_to_rename = ['visitors', 'day_of_week', 'holiday_flg', 'reserve_visitors', 'day_ind']
for i in range(1, k+1):
    a_shift = a[index_cols + cols_to_rename].copy()
    a_shift['ind'] = a_shift['ind'] - i
    foo = lambda x: '{}_lag_{}'.format(x, i) if x in cols_to_rename else x
    a_shift = a_shift.rename(columns=foo)
    b = pd.merge(b, a_shift, on=index_cols, how='left')
    

In [ ]:
def append_lag(grid, lag_begin, lag_length ):
    ''' 
    Add lag information to  grid to create training set
    Specifically, given a row with day_ind = D, and lag_begin = 7, lag_end = 14
    we add lag_7, lag_8, ..., lag_14 to this row   
    
    This is used to traing a model to forecast the visitors lag_begin days in the future
    '''
    index_cols = ['air_store_id', 'day_ind']
    cols_to_rename = ['visitors', 'day_of_week', 'holiday_flg', 'holiday_eve', 'closed', 'reserve_visitors']
    
    grid_cp = grid.copy()
    lag_end = lag_begin + lag_length - 1
    for day_shift in range(lag_begin, lag_end + 1):
        print('train day:', day_shift)
        grid_shift = grid[index_cols + cols_to_rename].copy()
        grid_shift['day_ind'] = grid_shift['day_ind'] + day_shift   
        foo = lambda x: '{}_lag_{}'.format(x, day_shift) if x in cols_to_rename else x
        grid_shift = grid_shift.rename(columns=foo)
        grid = pd.merge(grid, grid_shift, on=index_cols, how='left')
        del grid_shift
    grid_train = grid[~pd.isnull(grid['visitors_lag_' + str(lag_end)])].copy()
    grid_train = grid_train[grid_train['closed'] != 1]
    grid_train.drop(['day_ind', 'closed', 'hpg_store_id'], axis=1, inplace=True)

    
    max_day_ind = np.max(grid.day_ind)
    target_day_ind = max_day_ind + lag_begin
    grid_test = grid_cp[grid_cp.day_ind == max_day_ind].copy()
   # grid_test.drop(['mean_visitors', 'median_visitors', 'min_visitors', 'max_visitors', 'count_visitors'], axis=1, inplace = True)
    print('shape', grid_test.shape)

### step 0.5: add mean, median, min, max, count of (store, day_of_week)

In [12]:
mean_visit = air_visit_hist.groupby(['air_store_id', 'day_of_week'], as_index=False)['visitors'].mean().rename(columns={'visitors': 'mean_visitors'})

In [14]:
median_visit = air_visit_hist.groupby(['air_store_id', 'day_of_week'], as_index=False)['visitors'].median().rename(columns={'visitors': 'median_visitors'})

In [15]:
min_visit = air_visit_hist.groupby(['air_store_id', 'day_of_week'], as_index=False)['visitors'].min().rename(columns={'visitors': 'min_visitors'})

In [16]:
max_visit = air_visit_hist.groupby(['air_store_id', 'day_of_week'], as_index=False)['visitors'].max().rename(columns={'visitors': 'max_visitors'})

In [17]:
count_visit = air_visit_hist.groupby(['air_store_id', 'day_of_week'], as_index=False)['visitors'].count().rename(columns={'visitors': 'count_visitors'})

In [33]:
air_visit_hist = air_visit_hist.merge(mean_visit, on=['air_store_id', 'day_of_week'])
air_visit_hist = air_visit_hist.merge(median_visit, on=['air_store_id', 'day_of_week'])
air_visit_hist = air_visit_hist.merge(min_visit, on=['air_store_id', 'day_of_week'])
air_visit_hist = air_visit_hist.merge(max_visit, on=['air_store_id', 'day_of_week'])
air_visit_hist = air_visit_hist.merge(count_visit, on=['air_store_id', 'day_of_week'])

### step 1: add dates when a store is closed.

In [36]:
from itertools import product

In [37]:
def get_grid(k = 3):
    '''
    Keep the last k weeks of air_vist_hist, then for any store missing on any day,  create the corresponding 
    row with expacted valud 0
    '''
    last_train_day = max(air_visit_hist.day_ind)
    first_train_day = last_train_day - k * 7 + 1
    
    #filter into desire time frame
    hist1 = air_visit_hist[(air_visit_hist.day_ind >= first_train_day) & (air_visit_hist.day_ind <= last_train_day)].copy()
    all_stores = hist1.air_store_id.unique()
    all_days = [i for i in range(first_train_day, last_train_day+1)]
    
    #create store x day grid
    grid = np.array(list(product(*[all_stores, all_days])))
    grid = pd.DataFrame(grid, columns=['air_store_id', 'day_ind_str' ])
    grid['day_ind'] = grid.apply(lambda r: int(r['day_ind_str']), axis=1)
    grid.drop('day_ind_str', axis=1, inplace=True)
    
    # add visit information 
    all_data = grid.merge(hist1, how='left', on=['air_store_id', 'day_ind'])
    
    # add date type information
    all_data = all_data.merge(date_info, on='day_ind', suffixes=['_l', ''])
    drop_columns = [col for col in all_data.columns if col[-1] == 'l']
    all_data.drop(drop_columns, inplace=True, axis=1)
    
    # add store information
    all_data = all_data.merge(air_store_info, on = 'air_store_id', suffixes = ['_l', ''])
    #drop_columns = [col for col in all_data.columns if col[-1] == 'l'] + ['calendar_date', 'date', 'latitude', 'longitude', 'hpg_store_id']
    drop_columns = [col for col in all_data.columns if col[-1] == 'l'] + ['calendar_date', 'date', 'latitude', 'longitude', 'month_ind']
    
    all_data.drop(drop_columns, inplace=True, axis=1)
    
    # for those dates on which the visit informaiton of a store is missing, assume that it was closed abd with visit number 0
    all_data['closed'] = all_data.apply(lambda r: 1 if pd.isnull(r['visitors']) else 0, axis=1)
    all_data.fillna(0, inplace=True)
    return all_data
        
    
    

In [8]:
#grid = get_grid(k=5)

### 2.  create data frames with lag information

Given gap, create training set with lag_gap, lagp_(gap+1) ....

In [243]:
def append_lag(grid, lag_begin, lag_length ):
    ''' 
    Add lag information to  grid to create training set
    Specifically, given a row with day_ind = D, and lag_begin = 7, lag_end = 14
    we add lag_7, lag_8, ..., lag_14 to this row   
    
    This is used to traing a model to forecast the visitors lag_begin days in the future
    '''
    index_cols = ['air_store_id', 'day_ind']
    cols_to_rename = ['visitors', 'day_of_week', 'holiday_flg', 'holiday_eve', 'closed', 'reserve_visitors']
    
    grid_cp = grid.copy()
    lag_end = lag_begin + lag_length - 1
    for day_shift in range(lag_begin, lag_end + 1):
        print('train day:', day_shift)
        grid_shift = grid[index_cols + cols_to_rename].copy()
        grid_shift['day_ind'] = grid_shift['day_ind'] + day_shift   
        foo = lambda x: '{}_lag_{}'.format(x, day_shift) if x in cols_to_rename else x
        grid_shift = grid_shift.rename(columns=foo)
        grid = pd.merge(grid, grid_shift, on=index_cols, how='left')
        del grid_shift
    grid_train = grid[~pd.isnull(grid['visitors_lag_' + str(lag_end)])].copy()
    grid_train = grid_train[grid_train['closed'] != 1]
    grid_train.drop(['day_ind', 'closed', 'hpg_store_id'], axis=1, inplace=True)

    
    max_day_ind = np.max(grid.day_ind)
    target_day_ind = max_day_ind + lag_begin
    grid_test = grid_cp[grid_cp.day_ind == max_day_ind].copy()
   # grid_test.drop(['mean_visitors', 'median_visitors', 'min_visitors', 'max_visitors', 'count_visitors'], axis=1, inplace = True)
    print('shape', grid_test.shape)
    f = lambda x: '{}_lag_{}'.format(x, str(lag_begin)) if x in cols_to_rename else x
    grid_test = grid_test.rename(columns=f)
    grid_test['target_day_ind'] = grid_test['day_ind'] + lag_begin   
    for day_shift in range(lag_begin + 1, lag_end + 1):
        print('test day:', day_shift)
        grid_shift = grid_cp[grid_cp.day_ind == (max_day_ind - day_shift + lag_begin)][['air_store_id'] + cols_to_rename].copy()
        f = lambda x: '{}_lag_{}'.format(x, day_shift) if x in cols_to_rename else x
        grid_shift = grid_shift.rename(columns=f)
        grid_test = pd.merge(grid_test, grid_shift, on='air_store_id')
        del grid_shift
    grid_test.drop(['day_ind'], axis=1, inplace=True)
    grid_test = pd.merge(grid_test, date_info, left_on='target_day_ind', right_on = 'day_ind')
    grid_test.drop(['day_ind'], axis=1, inplace=True)
    grid_test = pd.merge(grid_test, air_reserve_day1, left_on = ['air_store_id', 'target_day_ind'],
                        right_on = ['air_store_id', 'day_ind'], how = 'left')
    grid_test.drop(['day_ind'], axis=1, inplace=True)
    grid_test = pd.merge(grid_test, hpg_reserve_day1, left_on = ['hpg_store_id', 'target_day_ind'],
                        right_on = ['hpg_store_id', 'day_ind'], suffixes = ['_air', '_hpg'], how = 'left')
    grid_test.fillna({'reserve_visitors_air': 0, 'reserve_visitors_hpg': 0}, inplace=True)
    grid_test['reserve_visitors'] = grid_test.apply(lambda r: r['reserve_visitors_air'] + r['reserve_visitors_hpg'], axis=1)
   
    grid_test.drop(['target_day_ind', 'calendar_date', 'date', 'day_ind', 'month_ind', 'reserve_visitors_air', 
                   'reserve_visitors_hpg', 'hpg_store_id',
                   'mean_visitors', 'median_visitors', 'min_visitors', 'max_visitors', 'count_visitors'], axis=1, inplace=True)   
    
    grid_test = grid_test.merge(mean_visit, on=['air_store_id', 'day_of_week'], how = 'left')
    grid_test = grid_test.merge(median_visit, on=['air_store_id', 'day_of_week'], how = 'left')
    grid_test = grid_test.merge(min_visit, on=['air_store_id', 'day_of_week'], how = 'left')
    grid_test = grid_test.merge(max_visit, on=['air_store_id', 'day_of_week'], how = 'left')
    grid_test = grid_test.merge(count_visit, on=['air_store_id', 'day_of_week'], how = 'left')
    grid_test.fillna({'mean_visitors': 0, 'median_visitors': 0, 'min_visitors':0, 'max_visitors': 0, 'count_visitors':0}, inplace=True)
    return grid_train, grid_test

### Label encode the categorical variables

In [40]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import LabelEncoder

In [11]:
class CatLabler(BaseEstimator, TransformerMixin):
    def __init__(self, cat_cols, lag_begin, lag_length):
        self.cat_cols = cat_cols
        self.lag_begin = lag_begin
        self.lag_length = lag_length
    def fit(self, X, y=None):
        encoders = {}
        self.weekday_encoder = LabelEncoder()
        self.weekday_encoder.fit(['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday'])
        for col in self.cat_cols:
            encoder = LabelEncoder()
            encoder.fit(X[col])
            encoders[col] = encoder
        self.encoders = encoders
        return self
    def transform(self, X, y=None):
        X_new = X.copy()
        for col in self.cat_cols:
            X_new[col] = self.encoders[col].transform(X[col])
        X_new['day_of_week'] = self.weekday_encoder.transform(X['day_of_week'])
        for i in range(self.lag_begin, self.lag_begin + self.lag_length):
            X_new['day_of_week_lag_'+str(i)] = self.weekday_encoder.transform(X['day_of_week_lag_'+str(i)])
        return X_new

In [41]:
genre_labler = LabelEncoder()
wday_labler = LabelEncoder()
store_labler = LabelEncoder()
area_labler = LabelEncoder()
wdays = ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']
genre_labler.fit(np.unique(air_store_info.air_genre_name))
wday_labler.fit(wdays)
store_labler.fit(np.unique(air_store_info.air_store_id))
area_labler.fit(np.unique(air_store_info.air_area_name))

LabelEncoder()

In [42]:
def encode_cat(X, lag_begin, lag_length):
    X_new = X.copy()
    X_new['day_of_week'] = wday_labler.transform(X['day_of_week'])
    X_new['air_store_id'] = store_labler.transform(X['air_store_id'])
    X_new['air_area_name'] = area_labler.transform(X['air_area_name'])
    X_new['air_genre_name'] = genre_labler.transform(X['air_genre_name'])
    for i in range(lag_begin, lag_begin + lag_length):
        X_new['day_of_week_lag_'+str(i)] = wday_labler.transform(X['day_of_week_lag_'+str(i)])
    return X_new

### Put all together to create final dataset

In [51]:
def create_train_test(k_grid=15, lag_length = 21, lag_begin0=1, lag_begin1=39):
    '''
    input:
    k_grid -- create k weeks grid starting from the last date in air_visit_hist
    k_train -- when creating the training data, if using the last_date in air_visit_hist as y, then X goes back to 
        last date - k_train * 7 day
        Example: if k_grid = 10 and lag_length = 21, the last day_ind in air_visit_hist is 477,
            then the grid will have every store with day_ind from 477 - 7*15 + 1 = 373 to 477.
        Now since lag_length = 21, then the training data with LAG = 2 will have the following:
        (i) day_ind = 477 as y, day_ind = 475, 474, ...., 475 - 21 + 1 = 475, 474, ..., 455 as X
        (ii) day_ind = 476 as y, day_ind = 474, 473, ....., 454 as X
        (iii) day_ind = 475 as y, day_ind = 473, 472, ...., 453 as X
         ....
         (x) day_ind = 464  as y, day_ind =  462                  408 as X
     
        The corresponding test/predict X_test and y_testwill be
        y_test for  day_ind = 477 + LAG = 479
        X_test consists of day_ind = 477, 476, ... 457
     
    lag: for each LAG between lag_begin0 and lag_begin1, create train set
    X_train_lag and y_train_lag where X_train has lag from LAG to 7 * k - 1,
    also create a test set X_test_lag 
    '''
    grid = get_grid(k_grid)
    last_train_day = np.max(air_visit_hist.day_ind)
    
    Data = {}
    Miss = {}
    for lag_begin in range(lag_begin0, lag_begin1 + 1):
        print('lag_begin=', lag_begin)
        lag_end = lag_begin + lag_length - 1
        gtrain, gtest = append_lag(grid, lag_begin, lag_length)
        cat_columns = ['air_store_id', 'air_genre_name', 'air_area_name']  
        #catLabler = CatLabler(cat_columns, lag_begin, lag_length)
       
        y_train = gtrain.visitors
        X_train0 = gtrain[gtest.columns]
        X_test0 = gtest
       
        X_train1 = encode_cat(X_train0, lag_begin, lag_length)
        X_test1 = encode_cat(X_test0, lag_begin, lag_length)
        strain = set(gtrain.air_store_id)
        stest = set(gtest.air_store_id)
        target_day = last_train_day + lag_begin     
        Miss[target_day] = stest.difference(strain)
        #catLabler.fit(X_train0)
       
        #X_train1 = catLabler.transform(X_train0)
      
        #X_test1 = catLabler.transform(X_test0)
       
        Data[lag_begin] = (y_train, X_train1, X_test1, X_test0.air_store_id)
    return Data, Miss

In [246]:
Data, Miss = create_train_test(k_grid=23, lag_length = 49, lag_begin0=1, lag_begin1=39)

lag_begin= 1
train day: 1
train day: 2
train day: 3
train day: 4
train day: 5
train day: 6
train day: 7
train day: 8
train day: 9
train day: 10
train day: 11
train day: 12
train day: 13
train day: 14
train day: 15
train day: 16
train day: 17
train day: 18
train day: 19
train day: 20
train day: 21
train day: 22
train day: 23
train day: 24
train day: 25
train day: 26
train day: 27
train day: 28
train day: 29
train day: 30
train day: 31
train day: 32
train day: 33
train day: 34
train day: 35
train day: 36
train day: 37
train day: 38
train day: 39
train day: 40
train day: 41
train day: 42
train day: 43
train day: 44
train day: 45
train day: 46
train day: 47
train day: 48
train day: 49
shape (827, 16)
test day: 2
test day: 3
test day: 4
test day: 5
test day: 6
test day: 7
test day: 8
test day: 9
test day: 10
test day: 11
test day: 12
test day: 13
test day: 14
test day: 15
test day: 16
test day: 17
test day: 18
test day: 19
test day: 20
test day: 21
test day: 22
test day: 23
test day: 24
tes

train day: 23
train day: 24
train day: 25
train day: 26
train day: 27
train day: 28
train day: 29
train day: 30
train day: 31
train day: 32
train day: 33
train day: 34
train day: 35
train day: 36
train day: 37
train day: 38
train day: 39
train day: 40
train day: 41
train day: 42
train day: 43
train day: 44
train day: 45
train day: 46
train day: 47
train day: 48
train day: 49
train day: 50
train day: 51
train day: 52
train day: 53
train day: 54
train day: 55
shape (827, 16)
test day: 8
test day: 9
test day: 10
test day: 11
test day: 12
test day: 13
test day: 14
test day: 15
test day: 16
test day: 17
test day: 18
test day: 19
test day: 20
test day: 21
test day: 22
test day: 23
test day: 24
test day: 25
test day: 26
test day: 27
test day: 28
test day: 29
test day: 30
test day: 31
test day: 32
test day: 33
test day: 34
test day: 35
test day: 36
test day: 37
test day: 38
test day: 39
test day: 40
test day: 41
test day: 42
test day: 43
test day: 44
test day: 45
test day: 46
test day: 47
test

train day: 41
train day: 42
train day: 43
train day: 44
train day: 45
train day: 46
train day: 47
train day: 48
train day: 49
train day: 50
train day: 51
train day: 52
train day: 53
train day: 54
train day: 55
train day: 56
train day: 57
train day: 58
train day: 59
train day: 60
train day: 61
shape (827, 16)
test day: 14
test day: 15
test day: 16
test day: 17
test day: 18
test day: 19
test day: 20
test day: 21
test day: 22
test day: 23
test day: 24
test day: 25
test day: 26
test day: 27
test day: 28
test day: 29
test day: 30
test day: 31
test day: 32
test day: 33
test day: 34
test day: 35
test day: 36
test day: 37
test day: 38
test day: 39
test day: 40
test day: 41
test day: 42
test day: 43
test day: 44
test day: 45
test day: 46
test day: 47
test day: 48
test day: 49
test day: 50
test day: 51
test day: 52
test day: 53
test day: 54
test day: 55
test day: 56
test day: 57
test day: 58
test day: 59
test day: 60
test day: 61
lag_begin= 14
train day: 14
train day: 15
train day: 16
train day:

train day: 58
train day: 59
train day: 60
train day: 61
train day: 62
train day: 63
train day: 64
train day: 65
train day: 66
train day: 67
shape (827, 16)
test day: 20
test day: 21
test day: 22
test day: 23
test day: 24
test day: 25
test day: 26
test day: 27
test day: 28
test day: 29
test day: 30
test day: 31
test day: 32
test day: 33
test day: 34
test day: 35
test day: 36
test day: 37
test day: 38
test day: 39
test day: 40
test day: 41
test day: 42
test day: 43
test day: 44
test day: 45
test day: 46
test day: 47
test day: 48
test day: 49
test day: 50
test day: 51
test day: 52
test day: 53
test day: 54
test day: 55
test day: 56
test day: 57
test day: 58
test day: 59
test day: 60
test day: 61
test day: 62
test day: 63
test day: 64
test day: 65
test day: 66
test day: 67
lag_begin= 20
train day: 20
train day: 21
train day: 22
train day: 23
train day: 24
train day: 25
train day: 26
train day: 27
train day: 28
train day: 29
train day: 30
train day: 31
train day: 32
train day: 33
train day:

test day: 38
test day: 39
test day: 40
test day: 41
test day: 42
test day: 43
test day: 44
test day: 45
test day: 46
test day: 47
test day: 48
test day: 49
test day: 50
test day: 51
test day: 52
test day: 53
test day: 54
test day: 55
test day: 56
test day: 57
test day: 58
test day: 59
test day: 60
test day: 61
test day: 62
test day: 63
test day: 64
test day: 65
test day: 66
test day: 67
test day: 68
test day: 69
test day: 70
test day: 71
test day: 72
test day: 73
lag_begin= 26
train day: 26
train day: 27
train day: 28
train day: 29
train day: 30
train day: 31
train day: 32
train day: 33
train day: 34
train day: 35
train day: 36
train day: 37
train day: 38
train day: 39
train day: 40
train day: 41
train day: 42
train day: 43
train day: 44
train day: 45
train day: 46
train day: 47
train day: 48
train day: 49
train day: 50
train day: 51
train day: 52
train day: 53
train day: 54
train day: 55
train day: 56
train day: 57
train day: 58
train day: 59
train day: 60
train day: 61
train day: 62


test day: 58
test day: 59
test day: 60
test day: 61
test day: 62
test day: 63
test day: 64
test day: 65
test day: 66
test day: 67
test day: 68
test day: 69
test day: 70
test day: 71
test day: 72
test day: 73
test day: 74
test day: 75
test day: 76
test day: 77
test day: 78
test day: 79
lag_begin= 32
train day: 32
train day: 33
train day: 34
train day: 35
train day: 36
train day: 37
train day: 38
train day: 39
train day: 40
train day: 41
train day: 42
train day: 43
train day: 44
train day: 45
train day: 46
train day: 47
train day: 48
train day: 49
train day: 50
train day: 51
train day: 52
train day: 53
train day: 54
train day: 55
train day: 56
train day: 57
train day: 58
train day: 59
train day: 60
train day: 61
train day: 62
train day: 63
train day: 64
train day: 65
train day: 66
train day: 67
train day: 68
train day: 69
train day: 70
train day: 71
train day: 72
train day: 73
train day: 74
train day: 75
train day: 76
train day: 77
train day: 78
train day: 79
train day: 80
shape (827, 16

test day: 76
test day: 77
test day: 78
test day: 79
test day: 80
test day: 81
test day: 82
test day: 83
test day: 84
test day: 85
lag_begin= 38
train day: 38
train day: 39
train day: 40
train day: 41
train day: 42
train day: 43
train day: 44
train day: 45
train day: 46
train day: 47
train day: 48
train day: 49
train day: 50
train day: 51
train day: 52
train day: 53
train day: 54
train day: 55
train day: 56
train day: 57
train day: 58
train day: 59
train day: 60
train day: 61
train day: 62
train day: 63
train day: 64
train day: 65
train day: 66
train day: 67
train day: 68
train day: 69
train day: 70
train day: 71
train day: 72
train day: 73
train day: 74
train day: 75
train day: 76
train day: 77
train day: 78
train day: 79
train day: 80
train day: 81
train day: 82
train day: 83
train day: 84
train day: 85
train day: 86
shape (827, 16)
test day: 39
test day: 40
test day: 41
test day: 42
test day: 43
test day: 44
test day: 45
test day: 46
test day: 47
test day: 48
test day: 49
test day: 5

In [248]:
pickle.dump(Data, open(data_dir + '/SubData23_49.p', 'wb'))
#pickle.dump(Miss, open(data_dir + '/Miss19_50.p', 'wb'))


MemoryError: 

## Submission 3.1 Xgboost

In [39]:
Data = pickle.load(open(data_dir + '/SubData15_21_d_2.p', 'rb'))

In [61]:
import xgboost as xgb

/home/alin/.pyenv/versions/3.6.3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [62]:
xgb_params = {
    'eta': 0.15,
    'max_depth': 5,
     'min_child_weight': 1,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'silent': 1
}

In [63]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [67]:
y_train, X_train, X_test, store = Data[36]

ly_train = np.log(y_train + 1)
X_train0, X_test0, ly_train0, ly_test0 = train_test_split(X_train, ly_train, test_size=0.33, random_state=42) 

xgtrain = xgb.DMatrix(X_train0.values, ly_train0)
xgtest = xgb.DMatrix(X_test0.values)

cvresult = xgb.cv(xgb_params, xgtrain, num_boost_round=1000, nfold=5, metrics='rmse', early_stopping_rounds=50, verbose_eval=False)

num_rounds = cvresult.shape[0] + 55
model = xgb.train(xgb_params, xgtrain, num_boost_round=num_rounds, evals = [(xgtrain, 'train')], verbose_eval=50)
    # make prediction
ly_predict = model.predict(xgtest)

mean_squared_error(ly_predict, ly_test0)

[0]	train-rmse:2.13085
[50]	train-rmse:0.494121
[100]	train-rmse:0.462974
[150]	train-rmse:0.440568
[200]	train-rmse:0.419467


0.26281886106566621

### try without day of week etc.

In [71]:
X_train1 = X_train0.copy()
X_test1 = X_test0.copy()

X_train1.drop(['reserve_visitors'], axis=1, inplace=True)
X_test1.drop(['reserve_visitors'], axis=1, inplace=True)


xgtrain1 = xgb.DMatrix(X_train1.values, ly_train0)
xgtest1 = xgb.DMatrix(X_test1.values)

cvresult1 = xgb.cv(xgb_params, xgtrain1, num_boost_round=1000, nfold=5, metrics='rmse', early_stopping_rounds=50, verbose_eval=False)

num_rounds = cvresult1.shape[0] + 55
model = xgb.train(xgb_params, xgtrain1, num_boost_round=num_rounds, evals = [(xgtrain1, 'train')], verbose_eval=50)
    # make prediction
ly_predict1 = model.predict(xgtest1)

print('mlse=',mean_squared_error(ly_predict1, ly_test0))

[0]	train-rmse:2.13085
[50]	train-rmse:0.510835
[100]	train-rmse:0.482686
[150]	train-rmse:0.456938
mlse= 0.289232914857


In [116]:
last_train_date = np.max(air_visit_hist.day_ind)
Results = []
xgb_params = {
    'eta': 0.15,
    'max_depth': 5,
     'min_child_weight': 1,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'silent': 1
}

for i in range(1, 2):
    print(i)
    y_train, X_train, X_test, stores = Data[i]
    ly_train = np.log(y_train + 1)
    xgtrain = xgb.DMatrix(X_train.values, ly_train)
    xgtest = xgb.DMatrix(X_test.values)
    cvresult = xgb.cv(xgb_params, xgtrain, num_boost_round=1000, nfold=5, metrics='rmse', early_stopping_rounds=50, verbose_eval=False)
    num_rounds = cvresult.shape[0] + 55
    model = xgb.train(xgb_params, xgtrain, num_boost_round=num_rounds, evals = [(xgtrain, 'train')], verbose_eval=50)
    # make prediction
    ly_predict = model.predict(xgtest)
    y_predict = np.exp(ly_predict) - 1
    rdf = pd.DataFrame({'id': stores, 'visitors': y_predict})
    dt = date_info[date_info.day_ind == (last_train_date + i)]
    rdf['id'] = rdf.apply(lambda r: r['id'] + '_' + dt.calendar_date, axis=1)
    Results.append(rdf)

1


NameError: name 'xgb' is not defined

In [74]:
Result_df = pd.concat(Results)

sample_submission = pd.read_csv(data_dir + '/sample_submission.csv.zip')
sample_submission['ord'] = sample_submission.index

output_df = Result_df.merge(sample_submission, on='id', suffixes=['','_r'])[['id', 'visitors', 'ord']]

output = output_df.sort_values('ord', axis=0)[['id', 'visitors']]
output.to_csv(data_dir + '/submission5_1.csv', index=False)

## Submission 3.2 lightgbm

In [57]:
import lightgbm as lgb

In [58]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [59]:
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression_l2',
    'metric': {'rmse'},
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.5
    }



In [252]:
d = 3

y_train, X_train_a, X_test_a, store = Data[d]
#remove_cols = ['holiday_eve'] + ['holiday_eve_lag_' + str(i) for i in range(d, d + 35)]
#remove_cols = ['air_store_id']
X_train = X_train_a.copy()
X_test = X_test_a.copy()
#X_train.drop(remove_cols, axis=1, inplace=True)
#X_test.drop(remove_cols, axis=1, inplace=True)

ly_train = np.log(y_train + 1)
X_train0, X_test0, ly_train0, ly_test0 = train_test_split(X_train, ly_train, test_size=0.33, random_state=42) 

lgb_train = lgb.Dataset(X_train0, ly_train0)

cv_lgb = lgb.cv(params, lgb_train, num_boost_round = 1000, nfold=5, stratified=False, early_stopping_rounds = 50, 
            verbose_eval = False)
num_rounds = len(cv_lgb['rmse-mean']) + 50
gbm = lgb.train(params,lgb_train,num_boost_round=num_rounds)
ly_predict = gbm.predict(X_test0)
#0: 0.24386
#1: 0.2337
print('mlse=', mean_squared_error(ly_predict, ly_test0))

mlse= 0.223088727748


In [23]:
X_train1 = X_train0.copy()
X_test1 = X_test0.copy()

X_train1.drop(['day_of_week', 'holiday_flg', 'holiday_eve'], axis=1, inplace=True)
X_test1.drop(['day_of_week', 'holiday_flg', 'holiday_eve'], axis=1, inplace=True)

lgb_train = lgb.Dataset(X_train1, ly_train0)

cv_lgb = lgb.cv(params, lgb_train, num_boost_round = 1000, nfold=5, stratified=False, early_stopping_rounds = 50, 
            verbose_eval = False)
num_rounds = len(cv_lgb['rmse-mean']) + 50
gbm = lgb.train(params,lgb_train,num_boost_round=num_rounds)
ly_predict = gbm.predict(X_test0)
print('mlse=', mean_squared_error(ly_predict, ly_test0))


mlse= 0.26846038696


In [155]:
Data1 = pickle.load(open(data_dir + '/SubData15_21_d_4.p', 'rb'))

In [254]:
last_train_date = np.max(air_visit_hist.day_ind)
Results = []
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression_l2',
    'metric': {'rmse'},
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9
    }

for i in range(1, 40):
    print(i)
    y_train, X_train, X_test, stores = Data[i]
    
   # drop_col = ['count_visitors']
   # X_train = X_train0.drop(drop_col, axis=1)
   # X_test = X_test0.drop(drop_col, axis=1)
    ly_train = np.log(y_train + 1)
    lgb_train = lgb.Dataset(X_train, ly_train)
    cv_lgb = lgb.cv(params, lgb_train, num_boost_round = 1000, nfold=5, stratified=False, early_stopping_rounds = 50, 
                verbose_eval = False)
    num_rounds = len(cv_lgb['rmse-mean']) + 50
    gbm = lgb.train(params,lgb_train,num_boost_round=num_rounds)
    ly_predict = gbm.predict(X_test)
    y_predict = np.exp(ly_predict) - 1
    rdf = pd.DataFrame({'id': stores, 'visitors': y_predict})
    dt = date_info[date_info.day_ind == (last_train_date + i)]
    rdf['id'] = rdf.apply(lambda r: r['id'] + '_' + dt.calendar_date, axis=1)
    Results.append(rdf)
    #print(rdf[rdf.id == 'air_00a91d42b08b08d9_2017-04-23'])

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39


In [257]:
Result_df = pd.concat(Results)

sample_submission = pd.read_csv(data_dir + '/sample_submission.csv.zip')
sample_submission['ord'] = sample_submission.index

output_df = Result_df.merge(sample_submission, on='id', suffixes=['','_r'])[['id', 'visitors', 'ord']]

output = output_df.sort_values('ord', axis=0)[['id', 'visitors']]
output.to_csv(data_dir + '/submission6_1.csv', index=False)

## try combine submission5_2_cd and submission6_1

In [258]:
output_cd = pd.read_csv(data_dir + '/submission5_2_cd.csv')

In [260]:
output_6 = pd.read_csv(data_dir + '/submission6_1.csv')

In [262]:
output = pd.merge(output_cd, output_6, on='id', suffixes=['_d', '_6'])

In [264]:
output['visitors'] = output.apply(lambda r: (r['visitors_d'] + r['visitors_6']) / 2.0, axis=1)

In [267]:
output.drop(['visitors_6', 'visitors_d'], axis=1, inplace = True)

In [269]:
output.to_csv(data_dir + '/submission6_2.csv', index=False)

In [40]:
output_dc['visitors'] = output_dc.apply(lambda r: int(r['visitors']) if r['visitors'] - int(r['visitors']) < 0.5 else int(r['visitors'])+1, axis=1)

In [42]:
output_dc.to_csv(data_dir + '/submission5_2_cdr.csv', index=False)

### try combine submission 5_2_c, 5_2_c and 6_1

In [270]:
output = pd.merge(output_cd, output_6, on='id', suffixes=['_d', '_6'])

output['visitors'] = output.apply(lambda r: (r['visitors_d']*2/3 + r['visitors_6']/3), axis=1)

output.drop(['visitors_6', 'visitors_d'], axis=1, inplace = True)

output.to_csv(data_dir + '/submission6_3.csv', index=False)

### 1/3 * (5_2_cd) + 2/3 * 6_1

In [277]:
output = pd.merge(output_cd, output_6, on='id', suffixes=['_d', '_6'])

output['visitors'] = output.apply(lambda r: (r['visitors_d']/3 + r['visitors_6']*2/3), axis=1)

output.drop(['visitors_6', 'visitors_d'], axis=1, inplace = True)

output.to_csv(data_dir + '/submission6_4.csv', index=False)